In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train=pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.drop(['id','url_legal','license','standard_error'],axis=1,inplace=True)
test.drop(['id','url_legal','license'],axis=1,inplace=True)
train.head(2)

In [ ]:
test.head(2)

In [ ]:
train['excerpt'][0]

In [ ]:
print(train['excerpt'][0].split(' '))

In [ ]:
train.isnull().sum(),test.isnull().sum()

In [ ]:
X=train.drop('target',axis=1)
y=train['target']

In [ ]:
X.shape

# Data Preropcessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
lemma=WordNetLemmatizer()

In [ ]:
X['excerpt'][0]

In [ ]:
test['excerpt'][0]

In [ ]:
def preprocess(text):
    corpus=[]
    
    for i in range(0,len(text)):
        read=re.sub('[^a-zA-Z]',' ',text['excerpt'][i])
        read=read.lower()
        read=read.split()
        read=[lemma.lemmatize(word) for word in read if not word in stopwords.words('english')]
        read=' '.join(read)
        corpus.append(read)
    
    return corpus

In [ ]:
X_corpus=preprocess(X)
X_corpus[:20]

In [ ]:
test_corpus=preprocess(test)
test_corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect=CountVectorizer().fit(X_corpus)

In [ ]:
print(vect.get_feature_names()[::200])

In [ ]:
X_vectorized=vect.transform(X_corpus)

In [ ]:
print(X_vectorized)

In [ ]:
from sklearn.linear_model import LinearRegression, PassiveAggressiveRegressor

model_lr_1=LinearRegression()
model_lr_1.fit(X_vectorized, y)

In [ ]:
predictions_lr_1=model_lr_1.predict(vect.transform(test_corpus))

In [ ]:
submission=pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission.head()

In [ ]:
predictions_lr_1[:5]

In [ ]:
submission.shape

In [ ]:
test.shape

In [ ]:
submission['my_target']=model_lr_1.predict(vect.transform(test_corpus))

In [ ]:
submission

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
pac_1=PassiveAggressiveRegressor()

In [ ]:
pac_1.fit(X_vectorized, y)

In [ ]:
submission['my_target']=pac_1.predict(vect.transform(test_corpus))
submission.head()

In [ ]:
mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_1=RandomForestRegressor(max_depth=5)

In [ ]:
rf_1.fit(X_vectorized, y)

In [ ]:
submission['my_target']=rf_1.predict(vect.transform(test_corpus))
mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect=TfidfVectorizer(min_df=5).fit(X_corpus)
len(vect.get_feature_names())

In [ ]:
X_vectorized=vect.transform(X_corpus)

In [ ]:
model_lr_2=LinearRegression()
model_lr_2.fit(X_vectorized, y)

In [ ]:
submission['my_target']=model_lr_2.predict(vect.transform(test_corpus))
mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
pac_2=PassiveAggressiveRegressor()
pac_2.fit(X_vectorized, y)
submission['my_target']=pac_2.predict(vect.transform(test_corpus))
mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
rf_2=RandomForestRegressor(max_depth=5)
rf_2.fit(X_vectorized, y)

In [ ]:
submission['my_target']=rf_2.predict(vect.transform(test_corpus))
mean_squared_error(submission['target'], submission['my_target'], squared=False)

In [ ]:
submission.drop(['target','my_target'],axis=1,inplace=True)
submission.head()

In [ ]:
rf_1.fit(X_vectorized, y)
submission['target']=rf_1.predict(vect.transform(test_corpus))
submission.to_csv('submission.csv',index=False)